Outline

    initialization
        create 1D array of agents (X, Y, angle, W)
        create 2D/3D trace grid (empty)
        create 2D/3D dual deposit grid (empty)

    simulation
        update state
        draw GUI
        process data (kernel inputs: data, deposit)
        propagation step (kernel inputs: agents, trace, deposit)
        relaxation step (kernel inputs: deposit)
        relaxation step (kernel inputs: trace)
        generate visualization (kernel inputs: deposit, trace, vis buffer)
        update GUI

    store grids
    tidy up

In [1]:
import numpy as np
from enum import IntEnum
from numpy.random import default_rng
import time
from datetime import datetime
import matplotlib.pyplot as plt
import taichi as ti

## Default root directory
ROOT = 'c:/UCSC/Projects/PolyPhy/repo/PolyPhy/'
# INPUT_FILE = ''
INPUT_FILE = ROOT + 'data/csv/sample_2D_linW.csv'
# INPUT_FILE = ROOT + 'data/csv/sample_2D_logW.csv'

## Type aliases
FLOAT_CPU = np.float32
INT_CPU = np.int32
FLOAT_GPU = ti.f32
INT_GPU = ti.i32

VEC2i = ti.types.vector(2, INT_GPU)
VEC3i = ti.types.vector(2, INT_GPU)
VEC2f = ti.types.vector(2, FLOAT_GPU)
VEC3f = ti.types.vector(3, FLOAT_GPU)

## Sampling strategy for agent directions
class EnumDirectionalSamplingType(IntEnum):
    DETERMINISTIC = 0
    STOCHASTIC = 1

## Deposit fetching strategy
class EnumDepositFetchingStrategy(IntEnum):
    NN = 0
    NN_PERTURBED = 1

## Initializations
ti.init(arch=ti.gpu, dynamic_index=True)
rng = default_rng()

[Taichi] version 1.0.3, llvm 10.0.0, commit fae94a21, win, python 3.6.12
[Taichi] Starting on arch=cuda


In [2]:
## Simulation-wide constants
N_DATA_DEFAULT = 1000
N_AGENTS_DEFAULT = 1000000
DOMAIN_SIZE_DEFAULT = (100.0, 100.0)
TRACE_RESOLUTION_MAX = 1024
DEPOSIT_DOWNSCALING_FACTOR = 1
STEERING_RATE = 0.5
MAX_DEPOSIT = 2.0
DOMAIN_MARGIN = 0.05

## State flags
directional_sampling_type = EnumDirectionalSamplingType.STOCHASTIC
deposit_fetching_strategy = EnumDepositFetchingStrategy.NN_PERTURBED

In [3]:
## Initialize data and agents
data = None
DOMAIN_MIN = None
DOMAIN_MAX = None
DOMAIN_SIZE = None
N_DATA = None
N_AGENTS = None
if len(INPUT_FILE) > 0:
    data = np.loadtxt(INPUT_FILE, delimiter=",").astype(FLOAT_CPU)
    N_DATA = data.shape[0]
    N_AGENTS = N_AGENTS_DEFAULT
    domain_min = (np.min(data[:,0]), np.min(data[:,1]))
    domain_max = (np.max(data[:,0]), np.max(data[:,1]))
    domain_size = np.subtract(domain_max, domain_min)
    DOMAIN_MIN = (domain_min[0] - DOMAIN_MARGIN * domain_size[0], domain_min[1] - DOMAIN_MARGIN * domain_size[1])
    DOMAIN_MAX = (domain_max[0] + DOMAIN_MARGIN * domain_size[0], domain_max[1] + DOMAIN_MARGIN * domain_size[1])
    DOMAIN_SIZE = np.subtract(DOMAIN_MAX, DOMAIN_MIN)
else:
    N_DATA = N_DATA_DEFAULT
    N_AGENTS = N_AGENTS_DEFAULT
    DOMAIN_SIZE = DOMAIN_SIZE_DEFAULT
    DOMAIN_MIN = (0.0, 0.0)
    DOMAIN_MAX = DOMAIN_SIZE_DEFAULT
    data = np.zeros(shape=(N_DATA, 3), dtype = FLOAT_CPU)
    data[:, 0] = rng.normal(loc = DOMAIN_MIN[0] + 0.5 * DOMAIN_MAX[0], scale = 0.15 * DOMAIN_SIZE[0], size = N_DATA)
    data[:, 1] = rng.normal(loc = DOMAIN_MIN[1] + 0.5 * DOMAIN_MAX[1], scale = 0.15 * DOMAIN_SIZE[1], size = N_DATA)
    data[:, 2] = 1.0

## Derived constants
DATA_TO_AGENTS_RATIO = FLOAT_CPU(N_DATA) / FLOAT_CPU(N_AGENTS)
DOMAIN_SIZE_MAX = np.max([DOMAIN_SIZE[0], DOMAIN_SIZE[1]])
TRACE_RESOLUTION = INT_CPU((FLOAT_CPU(TRACE_RESOLUTION_MAX) * DOMAIN_SIZE[0] / DOMAIN_SIZE_MAX, FLOAT_CPU(TRACE_RESOLUTION_MAX) * DOMAIN_SIZE[1] / DOMAIN_SIZE_MAX))
DEPOSIT_RESOLUTION = (TRACE_RESOLUTION[0] // DEPOSIT_DOWNSCALING_FACTOR, TRACE_RESOLUTION[1] // DEPOSIT_DOWNSCALING_FACTOR)
VIS_RESOLUTION = TRACE_RESOLUTION

agents = np.zeros(shape=(N_AGENTS, 4), dtype = FLOAT_CPU)
agents[:, 0] = rng.uniform(low = DOMAIN_MIN[0], high = DOMAIN_MAX[0], size = N_AGENTS)
agents[:, 1] = rng.uniform(low = DOMAIN_MIN[1], high = DOMAIN_MAX[1], size = N_AGENTS)
agents[:, 2] = rng.uniform(low = 0.0, high = 2.0 * np.pi, size = N_AGENTS)
agents[:, 3] = 1.0

print('Simulation domain min:', DOMAIN_MIN)
print('Simulation domain max:', DOMAIN_MAX)
print('Simulation domain size:', DOMAIN_SIZE)
print('Trace grid resolution:', TRACE_RESOLUTION)
print('Deposit grid resolution:', DEPOSIT_RESOLUTION)
print('Data sample:', data[0, :])
print('Agent sample:', agents[0, :])
print('Number of agents:', N_AGENTS)
print('Number of data points:', N_DATA)

Simulation domain min: (-457.9745269775391, -497.59637451171875)
Simulation domain max: (445.08982238769534, 6.09272575378418)
Simulation domain size: [903.06434937 503.68910027]
Trace grid resolution: [1024  571]
Deposit grid resolution: (1024, 571)
Data sample: [ -97.13328  -316.2591      7.398269]
Agent sample: [-145.52307     -75.153915      0.68970186    1.        ]
Number of agents: 1000000
Number of data points: 10000


In [4]:
## Allocate GPU memory fields
data_field = ti.Vector.field(n = 3, dtype = FLOAT_GPU, shape = N_DATA)
agents_field = ti.Vector.field(n = 4, dtype = FLOAT_GPU, shape = N_AGENTS)
deposit_field = ti.Vector.field(n = 2, dtype = FLOAT_GPU, shape = DEPOSIT_RESOLUTION)
trace_field = ti.Vector.field(n = 1, dtype = FLOAT_GPU, shape = TRACE_RESOLUTION)
vis_field = ti.Vector.field(n = 3, dtype = FLOAT_GPU, shape = VIS_RESOLUTION)
print('Total GPU memory allocated:', INT_CPU(4 * (\
    data_field.shape[0] * 3 + \
    agents_field.shape[0] * 4 + \
    deposit_field.shape[0] * deposit_field.shape[1] * 2 + \
    trace_field.shape[0] * trace_field.shape[1] * 1 + \
    vis_field.shape[0] * vis_field.shape[1] * 3 \
    ) / 2 ** 20), 'MB')

Total GPU memory allocated: 28 MB


In [5]:
## Define all GPU kernels and functions
@ti.kernel
def zero_field(f: ti.template()):
    for cell in ti.grouped(f):
        f[cell].fill(0.0)
    return

@ti.kernel
def copy_field(dst: ti.template(), src: ti.template()): 
    for cell in ti.grouped(dst):
        dst[cell] = src[cell]
    return

@ti.func
def world_to_grid_2D(pos_world, domain_min, domain_max, grid_resolution) -> VEC2i:
    pos_relative = (pos_world - domain_min) / (domain_max - domain_min)
    grid_coord = ti.cast(pos_relative * ti.cast(grid_resolution, FLOAT_GPU), INT_GPU)
    return ti.max(VEC2i(0, 0), ti.min(grid_coord, grid_resolution))

@ti.func
def angle_to_dir_2D(angle) -> VEC2f:
    return VEC2f(ti.cos(angle), ti.sin(angle))

@ti.func
def custom_mod(a, b) -> FLOAT_GPU:
    return a - b * ti.floor(a / b)

@ti.kernel
def data_step(data_deposit: FLOAT_GPU, current_deposit_index: INT_GPU):
    for point in ti.ndrange(data_field.shape[0]):
        pos = VEC2f(0.0, 0.0)
        pos[0], pos[1], weight = data_field[point]
        deposit_cell = world_to_grid_2D(pos, VEC2f(DOMAIN_MIN), VEC2f(DOMAIN_MAX), VEC2i(DEPOSIT_RESOLUTION))
        deposit_field[deposit_cell][current_deposit_index] += data_deposit * weight
    return

@ti.kernel
def agent_step(sense_distance: FLOAT_GPU,\
               sense_angle: FLOAT_GPU,\
               steering_rate: FLOAT_GPU,\
               step_size: FLOAT_GPU,\
               agent_deposit: FLOAT_GPU,\
               current_deposit_index: INT_GPU,\
               directional_sampling_type: INT_GPU,\
               deposit_fetching_strategy: INT_GPU):
    for agent in ti.ndrange(agents_field.shape[0]):
        pos = VEC2f(0.0, 0.0)
        pos[0], pos[1], angle, weight = agents_field[agent]
        
        dir_fwd = angle_to_dir_2D(angle)
        angle_mut = angle + (ti.random(dtype=FLOAT_GPU) - 0.5) * sense_angle
        dir_mut = angle_to_dir_2D(angle_mut)

        deposit_fwd = 0.0
        deposit_mut = 0.0
        if deposit_fetching_strategy == EnumDepositFetchingStrategy.NN:
            deposit_fwd = deposit_field[world_to_grid_2D(pos + sense_distance * dir_fwd, VEC2f(DOMAIN_MIN), VEC2f(DOMAIN_MAX), VEC2i(DEPOSIT_RESOLUTION))][current_deposit_index]
            deposit_mut = deposit_field[world_to_grid_2D(pos + sense_distance * dir_mut, VEC2f(DOMAIN_MIN), VEC2f(DOMAIN_MAX), VEC2i(DEPOSIT_RESOLUTION))][current_deposit_index]
        elif deposit_fetching_strategy == EnumDepositFetchingStrategy.NN_PERTURBED:
            field_dd = 2.0 * ti.cast(DOMAIN_SIZE[0], FLOAT_GPU) / ti.cast(DEPOSIT_RESOLUTION[0], FLOAT_GPU)
            pos_fwd = pos + sense_distance * dir_fwd + (field_dd * ti.random(dtype=FLOAT_GPU) * angle_to_dir_2D(360.0 * ti.random(dtype=FLOAT_GPU)))
            deposit_fwd = deposit_field[world_to_grid_2D(pos_fwd, VEC2f(DOMAIN_MIN), VEC2f(DOMAIN_MAX), VEC2i(DEPOSIT_RESOLUTION))][current_deposit_index]
            pos_mut = pos + sense_distance * dir_mut + (field_dd * ti.random(dtype=FLOAT_GPU) * angle_to_dir_2D(360.0 * ti.random(dtype=FLOAT_GPU)))
            deposit_mut = deposit_field[world_to_grid_2D(pos_mut, VEC2f(DOMAIN_MIN), VEC2f(DOMAIN_MAX), VEC2i(DEPOSIT_RESOLUTION))][current_deposit_index]

        angle_new = 0.0
        if directional_sampling_type == EnumDirectionalSamplingType.DETERMINISTIC:
            angle_new = (angle) if (deposit_fwd > deposit_mut) else (steering_rate * angle_mut + (1.0-steering_rate) * angle)
        elif directional_sampling_type == EnumDirectionalSamplingType.STOCHASTIC:
            angle_new = (angle) if (ti.random(dtype=FLOAT_GPU) < deposit_fwd / (deposit_fwd + deposit_mut)) else (steering_rate * angle_mut + (1.0-steering_rate) * angle)
        dir_new = angle_to_dir_2D(angle_new)
        pos_new = pos + step_size * dir_new

        pos_new[0] = custom_mod(pos_new[0] - DOMAIN_MIN[0], DOMAIN_SIZE[0]) + DOMAIN_MIN[0]
        pos_new[1] = custom_mod(pos_new[1] - DOMAIN_MIN[1], DOMAIN_SIZE[1]) + DOMAIN_MIN[1]

        agents_field[agent][0] = pos_new[0]
        agents_field[agent][1] = pos_new[1]
        agents_field[agent][2] = angle_new

        deposit_cell = world_to_grid_2D(pos_new, VEC2f(DOMAIN_MIN), VEC2f(DOMAIN_MAX), VEC2i(DEPOSIT_RESOLUTION))
        deposit_field[deposit_cell][current_deposit_index] += agent_deposit * weight

        trace_cell = world_to_grid_2D(pos_new, VEC2f(DOMAIN_MIN), VEC2f(DOMAIN_MAX), VEC2i(TRACE_RESOLUTION))
        trace_field[trace_cell][0] += ti.cast(N_DATA, FLOAT_GPU) / ti.cast(N_AGENTS, FLOAT_GPU) * weight
    return

DIFFUSION_KERNEL = [1.0, 1.0, 0.707]
DIFFUSION_KERNEL_NORM = DIFFUSION_KERNEL[0] + 4.0 * DIFFUSION_KERNEL[1] + 4.0 * DIFFUSION_KERNEL[2]

@ti.kernel
def deposit_relaxation_step(attenuation: FLOAT_GPU, current_deposit_index: INT_GPU):
    for cell in ti.grouped(deposit_field):
        value =  DIFFUSION_KERNEL[0] * deposit_field[cell + ( 0, 0)][current_deposit_index]\
               + DIFFUSION_KERNEL[1] * deposit_field[cell + (-1, 0)][current_deposit_index]\
               + DIFFUSION_KERNEL[1] * deposit_field[cell + ( 1, 0)][current_deposit_index]\
               + DIFFUSION_KERNEL[1] * deposit_field[cell + ( 0,-1)][current_deposit_index]\
               + DIFFUSION_KERNEL[1] * deposit_field[cell + ( 0, 1)][current_deposit_index]\
               + DIFFUSION_KERNEL[2] * deposit_field[cell + (-1,-1)][current_deposit_index]\
               + DIFFUSION_KERNEL[2] * deposit_field[cell + ( 1, 1)][current_deposit_index]\
               + DIFFUSION_KERNEL[2] * deposit_field[cell + ( 1,-1)][current_deposit_index]\
               + DIFFUSION_KERNEL[2] * deposit_field[cell + (-1, 1)][current_deposit_index]
        deposit_field[cell][1 - current_deposit_index] = attenuation * value / DIFFUSION_KERNEL_NORM
    return

@ti.kernel
def trace_relaxation_step(attenuation: FLOAT_GPU):
    for cell in ti.grouped(trace_field):
        trace_field[cell][0] *= attenuation
    return

@ti.kernel
def render_visualization(deposit_vis: FLOAT_GPU, trace_vis: FLOAT_GPU, current_deposit_index: INT_GPU):
    for x, y in ti.ndrange(vis_field.shape[0], vis_field.shape[1]):
        deposit_val = deposit_field[x * DEPOSIT_RESOLUTION[0] // VIS_RESOLUTION[0], y * DEPOSIT_RESOLUTION[1] // VIS_RESOLUTION[1]][current_deposit_index]
        trace_val = trace_field[x * TRACE_RESOLUTION[0] // VIS_RESOLUTION[0], y * TRACE_RESOLUTION[1] // VIS_RESOLUTION[1]]
        vis_field[x, y] = VEC3f(trace_vis * trace_val, deposit_vis * deposit_val, 0.0)
    return

In [6]:
## Initialize GPU fields
data_field.from_numpy(data)
agents_field.from_numpy(agents)
zero_field(deposit_field)
zero_field(trace_field)
zero_field(vis_field)

In [7]:
## Main simulation & vis loop
sense_distance = 2.0
sense_angle = 3.0
step_size = 0.2
deposit_attenuation = 0.9
trace_attenuation = 0.98
data_deposit = 0.1 * MAX_DEPOSIT
agent_deposit = data_deposit * DATA_TO_AGENTS_RATIO
deposit_vis = 1.0
trace_vis = 1.0

current_deposit_index = 0
data_edit_index = 0
do_simulate = True

window = ti.ui.Window('PolyPhy', (vis_field.shape[0], vis_field.shape[1]), show_window = True)
canvas = window.get_canvas()

def edit_data(edit_index: INT_CPU) -> INT_CPU:
    mouse_rel_pos = window.get_cursor_pos()
    mouse_pos = np.add(DOMAIN_MIN, np.multiply(mouse_rel_pos, DOMAIN_SIZE))
    data[edit_index, 0:2] = mouse_pos
    data_field.from_numpy(data)
    edit_index = (edit_index + 1) % N_DATA
    return edit_index

def stamp():
    return datetime.fromtimestamp(time.time()).strftime("%Y-%m-%d_%H-%M-%S")

while window.running:
    do_screenshot = False
    do_quit = False

    if window.get_event(ti.ui.PRESS):
        if window.event.key == 's': do_screenshot = True
        elif window.event.key in [ti.ui.ESCAPE]: do_quit = True
        elif window.event.key in [ti.ui.LMB]:
            data_edit_index = edit_data(data_edit_index)
    if window.is_pressed(ti.ui.RMB):
        data_edit_index = edit_data(data_edit_index)
    
    window.GUI.begin('Main', 0.01, 0.01, 0.32 * 1024.0 / FLOAT_CPU(VIS_RESOLUTION[0]), 0.46 * 1024.0 / FLOAT_CPU(VIS_RESOLUTION[1]))

    window.GUI.text("MCPM parameters:")
    sense_distance = window.GUI.slider_float('Sense dist', sense_distance, 0.1, 0.05 * np.max([DOMAIN_SIZE[0], DOMAIN_SIZE[1]]))
    sense_angle = window.GUI.slider_float('Sense angle', sense_angle, 0.1, 10.0)
    step_size = window.GUI.slider_float('Step size', step_size, 0.0, 0.002 * np.max([DOMAIN_SIZE[0], DOMAIN_SIZE[1]]))
    data_deposit = window.GUI.slider_float('Data deposit', data_deposit, 0.0, MAX_DEPOSIT)
    agent_deposit = window.GUI.slider_float('Agent deposit', agent_deposit, 0.0, MAX_DEPOSIT * FLOAT_CPU(N_DATA) / FLOAT_CPU(N_AGENTS))
    deposit_attenuation = window.GUI.slider_float('Deposit attn', deposit_attenuation, 0.8, 0.999)
    trace_attenuation = window.GUI.slider_float('Trace attn', trace_attenuation, 0.9, 0.999)
    deposit_vis = window.GUI.slider_float('Deposit vis', deposit_vis, 0.0, 1.0)
    trace_vis = window.GUI.slider_float('Trace vis', trace_vis, 0.0, 1.0)

    window.GUI.text("Directional sampling:")
    if window.GUI.checkbox("Deterministic", directional_sampling_type == EnumDirectionalSamplingType.DETERMINISTIC):
        directional_sampling_type = EnumDirectionalSamplingType.DETERMINISTIC
    if window.GUI.checkbox("Stochastic", directional_sampling_type == EnumDirectionalSamplingType.STOCHASTIC):
        directional_sampling_type = EnumDirectionalSamplingType.STOCHASTIC

    window.GUI.text("Deposit fetching:")
    if window.GUI.checkbox("Nearest neighbor", deposit_fetching_strategy == EnumDepositFetchingStrategy.NN):
        deposit_fetching_strategy = EnumDepositFetchingStrategy.NN
    if window.GUI.checkbox("Noise-perturbed NN", deposit_fetching_strategy == EnumDepositFetchingStrategy.NN_PERTURBED):
        deposit_fetching_strategy = EnumDepositFetchingStrategy.NN_PERTURBED

    window.GUI.text("Misc controls:")
    do_simulate = window.GUI.checkbox("Run simulation", do_simulate)
    do_screenshot = do_screenshot | window.GUI.button('Screenshot')
    do_quit = do_quit | window.GUI.button('Quit')
    window.GUI.end()

    if do_simulate:
        data_step(data_deposit, current_deposit_index)
        agent_step(sense_distance, sense_angle, STEERING_RATE, step_size, agent_deposit, current_deposit_index, directional_sampling_type, deposit_fetching_strategy)
        deposit_relaxation_step(deposit_attenuation, current_deposit_index)
        trace_relaxation_step(trace_attenuation)
        current_deposit_index = 1 - current_deposit_index

    render_visualization(deposit_vis, trace_vis, current_deposit_index)
    canvas.set_image(vis_field)

    if do_screenshot:
        window.write_image(ROOT + 'capture/screenshot_' + stamp() + '.png')
    window.show()
    if do_quit:
        break

window.destroy()

In [ ]:
## Store fits
current_stamp = stamp()
deposit = deposit_field.to_numpy()
np.save(ROOT + 'data/fits/deposit_' + current_stamp + '.npy', deposit)
trace = trace_field.to_numpy()
np.save(ROOT + 'data/fits/trace_' + current_stamp + '.npy', trace)

In [ ]:
## Plot results
plt.figure(figsize = (10.0, 10.0))
plt.imshow(np.transpose(deposit[:,:,0]))
plt.figure(figsize = (10.0, 10.0))
deposit_restored = np.load(ROOT + 'data/fits/deposit_' + current_stamp + '.npy')
plt.imshow(np.transpose(deposit_restored[:,:,0]))

plt.figure(figsize = (10.0, 10.0))
plt.imshow(np.transpose(trace[:,:,0]))
plt.figure(figsize = (10.0, 10.0))
trace_restored = np.load(ROOT + 'data/fits/trace_' + current_stamp + '.npy')
plt.imshow(np.transpose(trace_restored[:,:,0]))